In [ ]:
# see https://beepb00p.xyz/ipynb-singleline.html

def ldisplay_md(fmt, *args, **kwargs):
    from IPython.display import Markdown
    display(Markdown(fmt.format(
        *(f'${latex(x)}$' for x in args),
        **{k: f'${latex(v)}$' for k, v in kwargs.items()})
    ))
    
def as_text(thing):
    from IPython.core.interactiveshell import InteractiveShell # type: ignore
    plain_formatter = InteractiveShell.instance().display_formatter.formatters['text/plain']
    pp = plain_formatter(thing)
    lines = pp.splitlines()
    return lines

def vcpad(lines, height):
    width = len(lines[0])
    missing = height - len(lines)
    above = missing // 2
    below = missing - above
    return [' ' * width for _ in range(above)] + lines + [' ' * width for _ in range(below)]

# terminal and emacs can't display markdown, so we have to use that as a workaround
def mdisplay_plain(fmt, *args, **kwargs):
    import re
    from itertools import chain
    fargs   = [as_text(a) for a in args]
    fkwargs = {k: as_text(v) for k, v in kwargs.items()}

    height = max(len(x) for x in chain(fargs, fkwargs.values()))

    pargs   = [vcpad(a, height) for a in fargs]
    pkwargs = {k: vcpad(v, height) for k, v in fkwargs.items()}

    textpos = height // 2

    lines = []
    for h in range(height):
        largs   = [a[h] for a in pargs]
        lkwargs = {k: v[h] for k, v in pkwargs.items()}
        if h == textpos:
            fstr = fmt
        else:
            # we want to keep the formatting specifiers (stuff in curly braces and empty everything else)
            fstr = ""
            for e in re.finditer(r'{.*?}', fmt):
                fstr = fstr + " " * (e.start() - len(fstr))
                fstr += e.group()
        lines.append(fstr.format(*largs, **lkwargs))
    for line in lines:
        print(line.rstrip())

ldisplay = ldisplay_md

TODO say something blah blah
this is pretty simple stuff, but I've never seen it visualised anywhere and I think it would have helped me if it existed
TODO literate programming blogging blah blah

TODO something here https://youtu.be/p_di4Zn4wz4?t=1217

In [ ]:
# init_printing(use_latex='mathjax') # TODO not sure if necessary?
# TODO render line numbers

In [ ]:
%matplotlib inline
# PRIVATE TODO hmm what does %matplotlib notebook do? fucks up export though...
from sympy import *
from sympy import diff as D, Function as F
from sympy.solvers.ode import classify_sysode # PRIVATE TODO use that?


t = symbols('t', real=True) # time
q = F('q')(t) # position
v = F('v')(t) # velocity
p = F('p')(t) # [conjugate] momentum

Sadly, jupyter's capabilities for literate programing turned out to be poorer than I expected...
Basically, you can't split a function among several cells. (TODO perhaps hack it via CSS or something?)
So, over the course of the following function I'm gonna communicate with you via comments.

In [ ]:
# NOEXPORT TODO if we display side by side it might actually be ok
def solve_lagrangian(Lag):
    # So, we've got a Lagrangian:
    ldisplay("$L$ = {}", Lag)

    # NOEXPORT TODO P jeez, I can' even insert link properly!
    """
    To actually get motion it's way more convenient to solve Hamilton's equations.
    It's fairly easy to get them via something called Legendre transormation:
    https://en.wikipedia.org/wiki/Hamiltonian_mechanics#Calculating_a_Hamiltonian_from_a_Lagrangian    
    """
    
    p_of_v = Eq(p, Lag.diff(v))
    [sol] = solve(p_of_v, v)
    v_of_p = Eq(v, sol)
    ldisplay("$p$ over $v$: {}", p_of_v)
    ldisplay("$v$ over $p$: {}", v_of_p)
    
    # looks like variable replacement in Sympy is a bit more tedious than I expected
    H = (v * p_of_v.rhs - Lag).subs({v_of_p.lhs: v_of_p.rhs})
    ldisplay("$H$ = {}", H) # cool, we've got Hamiltonian
    
    dq_eq = Eq(q.diff(t),  H.diff(p))
    dp_eq = Eq(p.diff(t), -H.diff(q))
    
    ldisplay("Hamilton's equations: {}", [dq_eq, dp_eq])

    """
    lambdify is neat! It substitutes numpy equivalents for sympy functions,
    so we get decent performance.
    """
    dq = lambdify((q, p, t), dq_eq.rhs)
    dp = lambdify((q, p, t), dp_eq.rhs)
    
    """
    Ok, let's finally have a break from this function, the actual numerical solving
    can be extracted in a different function
    """
    return (dq, dp)
# NOEXPORT TODO might be interesting to pass q0v0?

Ok, now we need to define some slightly cryptic routines to integrate and plot stuff. Hopefully, later their purpose would become more clear.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9, 9]
from sympy.plotting import plot, plot_parametric
from scipy.integrate import odeint 
from numpy import linspace

# PRIVATE TODO might be interesting to pass initial velocity?
def plot_motion(
    Lag, 
    q0p0s=[], # initial data
    T_max=20, T_steps=1000, 
    q_max=None, 
    p_max=None,
    plot_pq=True,
    plot_phase=True,
    plot_legends=True,
):
    # hacky way for convenience of taking either symbolic Lagrangian or numeric equations
    dq, dp = Lag if isinstance(Lag, tuple) else solve_lagrangian(Lag)
    
    N = len(q0p0s)
    labels = [f'{float(qq):.1f}, {float(pp):.1f}' for (qq, pp) in q0p0s]
    
    times = linspace(0, T_max, T_steps)
    def integrate_step(qp, t):
        prev_q, prev_p = qp
        next_q = dq(prev_q, prev_p, t)
        next_p =  dp(prev_q, prev_p, t)
        return (next_q, next_p)
    
    all_ts = []
    all_qs = []
    all_ps = []
    for q0, p0 in q0p0s:
        sol = odeint(integrate_step, (q0, p0), times)
        all_ts.append(times)
        all_qs.append(sol[:, 0])
        all_ps.append(sol[:, 1])

    # ok, let's plot everything now!

    blues  = plt.cm.winter(linspace(0, 1, N))
    reds   = plt.cm.autumn(linspace(0, 1, N))
    greens = plt.cm.Set1(linspace(0.5, 1, N))
    # PRIVATE TODO ugh. better colors?

    if plot_pq:
        plt.figure(figsize=(7,7))
        plt.grid()
        plt.title('position/momentum over time')
        plt.xlabel('t')
        plt.ylabel('q,p')
        for label, ts, qs, ps, blue, red in zip(labels, all_ts, all_qs, all_ps, blues, reds):
            plt.plot(ts, qs, color=blue, label=f'q {label}')
            plt.plot(ts, ps, color=red, label=f'p {label}')
        if plot_legends: plt.legend(loc='upper left')
        plt.show()

    if plot_phase:
        plt.figure(figsize=(15,7))
        from mpl_toolkits.mplot3d import Axes3D
        ax = plt.subplot(1, 2, 1)
        ax3d = plt.subplot(1, 2, 2, projection='3d')
    
        ax.grid()
        ax.set_title('phase plot')
        ax.set_xlabel('q(t)')
        ax.set_ylabel('p(t)')
        if q_max is not None: ax.set_xlim((-q_max, q_max))
        if p_max is not None: ax.set_ylim((-p_max, p_max))
        for label, ts, qs, ps, green in zip(labels, all_ts, all_qs, all_ps, greens):
            ax.plot(qs, ps, color=green, label=f'{label}')
        if plot_legends: ax.legend(loc='upper left')

        ax3d.grid()
        ax3d.set_title('phase plot')
        ax3d.set_xlabel('q')
        ax3d.set_ylabel('p')
        ax3d.set_zlabel('t')
        if q_max is not None: ax3d.set_xlim((-q_max, q_max))
        if p_max is not None: ax3d.set_ylim((-p_max, p_max))
        for label, ts, qs, ps, green in zip(labels, all_ts, all_qs, all_ps, greens):
            ax3d.plot(qs, ps, ts, color=green, label=label)
        if plot_legends: ax3d.legend(loc='upper left')
        plt.show()

Ok, now that we defined the boilerplate, let's actually do something.
    
First test subject is pretty obvious: the harmonic oscillator.

In [ ]:
half = Rational(1, 2)  
L_harm = half * v ** 2 - half * q ** 2
plot_motion(
    L_harm, 
    q0p0s=[
        (-10, 10),
        (10 , 10),
        (10 , -10),
        (-10, -10),
    ]
)

As expected, we get circular motion in the position-momentum plane and a nice spiral on the projective plot.
Let's try something else?

In [ ]:
L_hill = half * v ** 2 + half * q ** 2
dqdp = solve_lagrangian(L_hill)
plot_motion(
    dqdp, 
    q0p0s=[
        (-1, -1),
        (1 , 1),
        (1 , -1),
        (-1, 1),
    ],
    T_max=2,
)

If you think about it, here we've got a potential hill! So the only equilibrium for this system is $q=0, p=0$. Within this Lagrangian, you can reach it if your momentum (which coincides with velocity) is larger than the position in absolute value and of the opposite sign.

TODO plot 3d lagrangian?

If the sign you your momentum is same as the sign of your position, you're basically forever doomed to slide down the potential! We can illustrate that by plotting more phase plots:

In [ ]:
def circle(R, phis):
    return [(R * sin(phi), R * cos(phi)) for phi in phis]

def circlespace(R, points):
    return circle(R, linspace(0, 2 * np.pi, points))
    
plot_motion(
    dqdp,
    q0p0s=circlespace(R=5.0, points=36),
    T_max=3,
    p_max=5, q_max=5,
    plot_pq=False, plot_legends=False,
)

If you look at the 2D phase plot, you can clearly spot hyperbolas. This is not a coincidence considering that the Hamiltonian can be interpreted as energy which is conserved during the motion, and the lines where $p^2 + q^2 = \text{const}$ are precisely hyperbolas!

Actually, matplotlib allows us to plot something even nicer:  `quiver` (usually called vector field or slope field) and `streamplot`.

In [ ]:
def plot_ham_field(
    Lag, 
    steps=30, 
    qlim=None, 
    plim=None,
    density=1,
):
    dq, dp = Lag if isinstance(Lag, tuple) else solve_lagrangian(Lag)
    
    qs = linspace(*qlim, steps)
    ps = linspace(*plim, steps)
    Q, P = np.meshgrid(qs, ps)
    
    T = 0 # we're exploiting knowledge that hamiltonians are time independent in our case
    # PRIVATE TODO plot some gifs and try time dependent Hamiltonians? Although not sure if there are any interesting examples in classical mechanics..
    Hq = dq(Q, P, T)
    Hp = dp(Q, P, T)
    
    fig, ax = plt.subplots()
    ax.set_title("Hamiltonian vector field")
    ax.set_xlabel('q')
    ax.set_ylabel('p')
    ax.set_aspect(aspect=1)
    # PRIVATE TODO set size as well?
    ax.streamplot(qs, ps, Hq, Hp, arrowstyle='-', density=density)
    ax.quiver(qs, ps, Hq, Hp, scale=100)
    plt.show()

In [ ]:
plot_ham_field(dqdp, qlim=(-7, 7), plim=(-7, 7))

In [ ]:
L = half * v ** 2 * q ** 2
# L = half * v ** 2 * (q ** 2 + 1)
# huh! this one behaves nicely... because it's not going to 0?

# TODO L = half * v ** 2 * q
# TODO hmmm. I used to have q^2. Can it be problematic for legendre transform?? Seems ok in terms of having inverse though
# TODO investigate it separately?? 
# TODO analytical solution https://www.wolframalpha.com/input/?i=q%27%3Dp%2Fq;+p%27%3Dp%5E2%2F+(2+*+q%5E2)
# doesn't seem to behave well for negative time. something has to be wrong, right?
dqdp = solve_lagrangian(L)
plot_motion(
    dqdp, 
    q0p0s=[
        (-1, -1),
        (1 , 1),
        (1 , -1),
        (-1, 1),
    ],
    T_max=0.5,
)

Now this is something fun! We get some warning from the ODE solver. I've chosen `T_max` carefully here, i you increase it past 0.5, the numerical solution just goes nuts, and fair enough, if you look at the Hamiltonian, when $q=0$ clearly something bad has to happen. Or does it? Sadly, Sympy has a bug in the solver for that type of equations, so let's try doing it the old way.

Starting with $\left [ \frac{d}{d t} q{\left (t \right )} = \frac{p{\left (t \right )}}{q^{2}{\left (t \right )}}, \quad \frac{d}{d t} p{\left (t \right )} = \frac{p^{2}{\left (t \right )}}{q^{3}{\left (t \right )}}\right ]$, if we divide LHS and RHS sides, we get something nice:

$$\frac{q'(t)}{p'(t)} = \frac{q(t)}{p(t)}$$
$$\frac{q'(t)}{q(t)} = \frac{p'(t)}{p(t)}$$
If we integrate both parts, we get
$$\ln q(t) = \ln p(t) + C$$
or, equivalently,
$$q(t) = C p(t)$$

I find it interesting that we kind of could have seen it coming from the Hamiltonian if we think about what does it take for it to be a constant along the trajectory. Not sure if this argument can be generally useful though. But the important bit is that in our case, we have $H = \frac{1}{2 C^2}$, so surely there can't be problem when $q = 0$. Let's dig further. If we substitute or result for $q$ in its Hamiltonian equation, we get:

$$C p'(t) = \frac{p(t)}{C^2 p^2(t)}$$
$$ p'(t) = \frac{1}{C^2} \frac{1}{p(t)}$$
$$p(t) = \pm \frac{1}{C^2} \sqrt{2} \sqrt{D + t}$$


First thing that I find interesting is that this clearly discriminates against negative $t$. TODO is that interesting actually?

Second, is that it's easy to see that when 

TODO ugh, must have something to do with branch cuts...

TODO shit. maybe just give up on this one and write something about non-regularity (as in, lagrangian is non-regular over the whole q = 0 line)

It's hard to tell what this Lagrangian represents TODO let me know.
TODO highligh todos in hakyll/ci?
TODO wtf?? slope field in wolfram is complete
https://www.wolframalpha.com/input/?i=q%27%3Dp%2Fq%5E2;+p%27%3Dp%5E2%2Fq%5E3
TODO mm. does it depend on which solution you choose or what?
TODO huh? wolfram is solving it in complex numbers... https://www.wolframalpha.com/input/?i=q%27%3Dp%2Fq%5E2;+p%27%3Dp%5E2%2Fq%5E3+,+q(0)%3D-5,+p(0)%3D5

TODO investigate what happens around 0...

In [ ]:
plot_ham_field(dqdp, qlim=(-1, 1), plim=(-1, 1))

Let's try something else

In [ ]:
L = exp(v) - q ** 2
dqdp = solve_lagrangian(L)
plot_motion(
    dqdp, 
    q0p0s=[
        (-1, -1),
        (1 , 1),
        (1 , -1),
        (-1, 1),
    ],
    T_max=0.5,
    plot_phase=False,
)

Uh-oh, momentum appears under logarigthm. Let's just exclude negative momentum for now..

In [ ]:
plot_motion(
    dqdp, 
    q0p0s=[(q0, 1) for q0 in np.linspace(-5, -0.5, 10)] + [(q0, 20) for q0 in np.linspace(-5, -0.5, 10)],
    T_max=2,
    plot_pq=False,
)
plot_ham_field(
    dqdp, 
    qlim=(-5, 5), plim=(0, 5),
    density=2,
)

TODO interesting! look at slope field, this lagrangian is always doomed!!!
TODO what if we continue it to complex plane?
TODO hmm. is the initial position the only thing defining when it dies?? TODO nah, se below, it seems pretty random..
TODO what happens to energy here?? Yep, exactly. when the energy is positive, it's fucked!
TODO figure out how this relates?
TODO it's a funny universe
TODO huh, x log x is 0 when x -> 0
basically, we need to find when slope of the phase plot is 0? so p'(t) = 0

It's interesting though that the flow begs for continuation past the $p = 0$ line. Quick speculation: if you take your momentum as purely complex, you can draw in TODO draw. Not sure if it has any interesting physical, mathematical consequences, so I guess I'll have to look at it some other time (leave visible TODO?).

Let's take a closer look at the stable phase orbits. TODO time flow?

In [ ]:
plot_motion(
    dqdp, 
    q0p0s=[(-0.2, p0) for p0 in np.linspace(1.0, 2.67, 15)],
    T_max=30,
    plot_pq=False,
)

TODO hmm. interesting that all orbits seem to evolve at the same rate? why is that? definitely need animation...


TODO summarise what's hard to do. also smth about lagrangians

In [ ]:
# TODO subsection for different lagrangians
# ok, type3 makes sense

# TODO exp + q ** 2?
# TODO exp + q? could be interesting...
# TODO right. any more interesting functions??

# L = log(v) - q ** 2

# ok, here it just bumps into the boundary as well
# TODO wonder why is it so similar to exp??
# TODO also, need to convert back to v from p?
# TODO hm. logarithmic dies as well, but only defined for q > 0; p > 0

# L = 1 / v ** 2  - q ** 2
# this one is weird. if it doesn't get to p = 0 singularity, it's fine and just keeps goint to infty
inits = [
    #(-q_i , -p_i),
    # (-5 , 5), # this guy lives a bit longer
    (-10 , -100),
    (-5 , -100),
    (-1 , -100),
    # (q_i  , p_i),
]

# TODO wtf?? what's up with duplication?

# TODO wolframs slope/direction plot is pretty cool. try to reproduce?
# TODO phase plot?
# TODO lambdify that and plot
# display stuff on the left?

"""
# TODO move this to body
Sadly, ironically Sympy can't really solve them symbolically appart from the simplest case.
https://github.com/sympy/sympy/issues/15707
"""

"""
s_q0, s_p0 = symbols('p_0 q_0', real=True)

sol = dsolve(
    system, 
    [q, p],
    # ics={
    #     q.subs(t, 0): s_q0,
    #     p.subs(t, 0): s_p0,
    # }
)
[q_sol, p_sol] = sol
display(q_sol, p_sol)

# TODO ugh. ok, I guess try numerically
(q_sol, p_sol) = (e.subs({s_q0: q0, s_p0: p0}) for e in (q_sol, p_sol))
# display(q_sol, p_sol)
#plot_parametric(
#    (q_sol.rhs, p_sol.rhs),
#    (t, 0, 10),
#    xlabel='q',
#    ylabel='p',
#)
# from sympy.utilities.lambdify import lambdify
"""